In [2]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
params = {'page':1, 'page_size':1}
response = sg.client.contactdb.recipients.count.get()
print(response.status_code)
print(response.body)
#print(response.headers)

200
b'{"recipient_count":1}\n'


In [25]:
# Sendgrid add recipients
data = [
    {"email": "jstenger@uvm.edu"}
]
response = sg.client.contactdb.recipients.post(request_body=data)
print(response.status_code)
print(response.body)

201
b'{"new_count":1,"updated_count":0,"error_count":0,"error_indices":[],"unmodified_indices":[],"persisted_recipients":["anN0ZW5nZXJAdXZtLmVkdQ=="],"errors":[]}\n'


In [45]:
# Sendgrid retrieve recipients
import json
params = {'page':1, 'page_size':1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
contacts = json.loads(response.body)
emails = [recipient['email'] for recipient in contacts['recipients']]
emails

200


['jstenger@uvm.edu']

In [7]:
response = sg.client.user.username.get()
print(response.status_code)
print(response.body)

200
b'{"user_id":1757996,"username":"bstenger"}'


parse index.xml to get latest feed item

In [9]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(feed.feed.title + ": " + newsletter.title)
print(newsletter.description)

newsletter: ADCN: A sports science and technology newsletter by Brad Stenger
<p>Welcome to my resurrected sports science and technology newsletter. I've given it a title, the "Athletes Data Community Newsletter," something that reflects my research interests.</p>
<p>I'm going into my last year of a Computer Science PhD at the University of Vermont and my primary research projects focus on privacy aspects of athletes’ health and performance data. Other papers in preparation: task analysis for athletes' data technologies based on field-based research, athletes' injury risk from an engineering systems perspective, socio-technical impediments and solutions for data sharing in sports. By putting athletes' data front and center I think that it will be possible to discuss the varied aspects of how data gets collected, managed and applied across the full spectrum of sports. At least I hope so.</p>
<section class="level3" id="concussions-in-2025">
<h3 class="anchored">Concussions in 2025</h3>
<

In [58]:
import re
from bs4 import BeautifulSoup
soup = BeautifulSoup(newsletter.description, 'html.parser')
sections = soup.find_all('section')
subject = '"subject": "' + newsletter.title + '"'
introduction = '"introduction":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in reversed(sections[0].find_previous_siblings('p'))]) + ']'
#section = '"sections":' + '[' + ",".join([ '{"section":"' + re.sub(r'"', r'\"', str(s)) + '"}' for s in sections]) + ']'
section = []
for s in sections:
    heading = '"heading": "' + re.sub(r'"', r'\"', str(s.find('h3'))) + '"'
    description = '"description": "' + str(s.find('p')) + '"'
    paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('p').find_next_siblings('p')]) + ']'
    sec = '{"section": {' + heading + ', ' + description + ', ' + paragraphs + '}}'
    section.append(sec)
h_sections = '"sections": [' + ",".join(section) + ']'
handlebar = '{' + subject + ', ' + introduction + ', ' + h_sections + '}'
print(handlebar)


{"subject": "ADCN: A sports science and technology newsletter by Brad Stenger", "introduction":[{"p":"<p>Welcome to my resurrected sports science and technology newsletter. I've given it a title, the \"Athletes Data Community Newsletter,\" something that reflects my research interests.</p>"},{"p":"<p>I'm going into my last year of a Computer Science PhD at the University of Vermont and my primary research projects focus on privacy aspects of athletes’ health and performance data. Other papers in preparation: task analysis for athletes' data technologies based on field-based research, athletes' injury risk from an engineering systems perspective, socio-technical impediments and solutions for data sharing in sports. By putting athletes' data front and center I think that it will be possible to discuss the varied aspects of how data gets collected, managed and applied across the full spectrum of sports. At least I hope so.</p>"}], "sections": [{"section": {"heading": "<h3 class=\"anchored

In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
from_email = Email("brad.stenger@gmail.com")
to_email = To("jstenger@uvm.edu")
subject = feed.feed.title + ": " + newsletter.title
content = Content(MimeType.html, newsletter.description)
mail = Mail(from_email, to_email, subject, content)
# template 
response = sg.client.mail.send.post(request_body=mail.get())
print(response.status_code)
print(response.body)
print(response.headers)

202
b''
Server: nginx
Date: Thu, 24 Jul 2025 19:52:43 GMT
Content-Length: 0
Connection: close
X-Message-Id: RXScMViKTOWfq0fFvi1hgA
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




In [ ]:
# Template ID: d-6afa3f23868e47d1821cae1fa52951a7 
